In [ ]:
#!jupyter nbconvert --to=python Sal_WOA_Comparison_python3.ipynb

# After changes in the script using Jupyter notebook, save the notebook, run this line, and clear the output.

In [ ]:
class SALcomp:
    '''
    class SALcomp(runname,resultpath,savepath,meshpath,ncpath,first_year,last_year,
                 WOAvar='s_an',mapproj='pc',savefig=False,get_overview = 'False',
                 use_temp_mask = 'False',layerwise=False)
                 
    Projection 'rob' (Robinson) results in strange values, please use only 'pc' Plate Carree!
    
    'use_temp_mask' loads additional FESOM 'temp' out put to use as a mask for the 'salt' output.
    '''
    def __init__(self,runname,resultpath,savepath,meshpath,ncpath,first_year,last_year,
                 WOAvar='s_an',
                 mapproj = 'pc',
                 savefig=False,
                 get_overview = False,
                 use_temp_mask = False,
                 layerwise=False,depth_array=[]):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.meshpath = meshpath
        self.ncpath = ncpath
        self.fyear = first_year
        self.lyear = last_year
        self.WOAvar = WOAvar
        self.mapproj = mapproj
        self.savefig = savefig
        self.get_overview = get_overview
        self.use_temp_mask = use_temp_mask
        self.layerwise = layerwise
        self.depth_array = depth_array

        import matplotlib.pyplot as plt
        import numpy as np
        #from netCDF4 import Dataset
        #from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        #import pickle

        import pyfesom2 as pf

        from load_interp_WOA_python3 import WOAdata
        from plot_Taylor_normalized import plt_Taylor_norm


        if(self.mapproj != 'pc'):
            raise ValueError('Except Plate Carree, other projection results in strange values, \nplease use only "pc", i.e. Plate Carree!')

        # load FESOM data -------------------------------------------------------------------------------------
        mesh       = pf.load_mesh(meshpath)
        years = [self.fyear, self.lyear]

        # check variables
        #NCfesom = self.resultpath + '/salt.'+self.runname+'.'+str(self.fyear)+'.nc'
        #!ncdump -h $NCfesom

        labelfesom = 'FESOM Salinity {0}-{1}'.format(self.fyear,self.lyear)
        unitfesom = 'Salinity [psu]' 

        # load data -------------------------------------------------------------------------------------
        SALfesom = pf.get_data(resultpath, "salt", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=False)

        # problem: sometimes FESOM salt output includes all gridpoints in deeper depth and not only ocean points!
        # idea: load FESOM temp output additionally to use it as mask
        # DOESN'T WORK !!! WHY???
        if(use_temp_mask == True):
            print('use_temp_mask = True\nLoad temperature data for masking...')
            TEMPfesom = pf.get_data(resultpath, "temp", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)
            SALfesom[TEMPfesom == 0] = 0 # apply mask
            print(np.shape(SALfesom))

            
        # load WOA data  -------------------------------------------------------------------------------------
        sal_int = WOAdata(self.runname,self.resultpath,self.meshpath,self.ncpath,self.WOAvar, get_overview=self.get_overview).woa_int    
        
        labelwoa = 'World Ocean Atlas Salinity'
        unitwoa = 'Salinity [psu]' 

        # apply sea mask to WOA as in FESOM ----------------------------------------------------------------------------------
        # assumption: there is no ocean where value in FESOM == 0
        sal_int_ma = np.copy(sal_int)
        sal_int_ma[SALfesom == 0] = 0

        # plot WOA and FESOM ----------------------------------------------------------------------------------
        if(self.layerwise):
            if(self.depth_array == []):
                self.depth_array = (0,50,200,1000,2000,4000)

            for d in self.depth_array:
                # get mesh index closest to desired depth
                i = pf.ind_for_depth(d,mesh) 
                # get midlevel depth
                plot_depth = str((mesh.zlev[i]+mesh.zlev[i+1])/2)

                if(True):
                    print('\nInput depth = {0}, plotting at depth = {1} m\nFESOM min = {2}, max = {3}\nWOA min = {4}, max = {5}'.format(
                        d,plot_depth,
                        np.nanmin(SALfesom[:,i]),np.nanmax(SALfesom[:,i]),
                        np.nanmin(sal_int_ma[:,i]),np.nanmax(sal_int_ma[:,i])))

                pf.plot(mesh, [sal_int_ma[:,i],SALfesom[:,i], sal_int_ma[:,i]-SALfesom[:,i]], 
                        rowscol= (1,3),
                        levels = (0,37,38),
                        units=unitwoa + '\n at depth = {0} m'.format(plot_depth), 
                        mapproj=self.mapproj, # robinson projection takes more time!
                        titles=[labelwoa, labelfesom, 'WOA - FESOM'],figsize = (20,20)
                       )

            # statistics  -------------------------------------------------------------------------------------
            # preparation of datasets
            if np.isnan(np.min(sal_int)): print('WARNING: The interpolated WOA field contains NaNs at depth')
            if np.isnan(np.min(SALfesom)): print('WARNING: The interpolated FESOM field contains NaNs at depth')


            for d in self.depth_array:
                # get mesh index closest to desired depth
                i = pf.ind_for_depth(d,mesh) 
                # get midlevel depth
                plot_depth = str((mesh.zlev[i]+mesh.zlev[i+1])/2)
                

                title = 'Taylor Diagram for Salinity at {0} m'.format(plot_depth)

                plt_Taylor_norm(sal_int[i,:],SALfesom[i,:], title=title)

            if(self.savefig==True): print('\n***\n***Too many figures to export...\n***')
        
        # mean over depth  -------------------------------------------------------------------------------------
        else:
            
            SALfesom_mean = np.nanmean(SALfesom, axis = 1)
            sal_int_ma_mean = np.nanmean(sal_int_ma, axis = 1)

            print('\nPlotting Salinity as mean over depth\nFESOM min = {0}, max = {1}\nWOA min = {2}, max = {3}'.format(
                    np.nanmin(SALfesom_mean),np.nanmax(SALfesom_mean),
                    np.nanmin(sal_int_ma_mean),np.nanmax(sal_int_ma_mean)))
    
            if True: #switch of large comparison plot only for testing single FESOM plot below!
                fig_data = pf.plot(mesh, [sal_int_ma_mean,SALfesom_mean, sal_int_ma_mean-SALfesom_mean], 
                        rowscol= (3,1),
                        levels = (0,37,38),
                        units=unitwoa, 
                        mapproj=self.mapproj, # robinson projection takes more time!
                        titles=['World Ocean Atlas: Interpolated Salinity for initialization', 
                                'FESOM Salinity ({0}-{1})'.format(first_year,last_year),
                                'Initialization fields minus FESOM'],
                        figsize = (20,20)
                       )
                # fig export  -------------------------------------------------------------------------------------
                if(self.savefig==True):
                    plt.savefig(self.savepath+self.runname+'_'+'Sal_WOA'+'_'+str(years[0])+'to'+str(years[1])+'.png', 
                            dpi = 300, bbox_inches='tight')
                plt.show(block=False)  
                
            ## just to test: plot only FESOM data
            if False: 
                fig_data = pf.plot(mesh, [SALfesom_mean], 
                    rowscol= (1,1),
                    levels = (0,37,38),
                    units=unitwoa, 
                    mapproj=self.mapproj, # robinson projection takes more time!
                    titles=['FESOM Salinity ({0}-{1})'.format(first_year,last_year)],
                    figsize = (20,20)
                   )
                plt.show(block=False)
                
            # statistics  -------------------------------------------------------------------------------------            
            # preparation of datasets
            if np.isnan(np.min(sal_int_ma_mean)): print('WARNING: The interpolated WOA field contains NaNs at depth')
            if np.isnan(np.min(SALfesom_mean)): print('WARNING: The interpolated FESOM field contains NaNs at depth')

            title = 'Taylor Diagram for Salinity \n(mean over depth)'
            plt_Taylor_norm(sal_int_ma_mean,SALfesom_mean,mask=True,title=title)
              
            # fig export  -------------------------------------------------------------------------------------
            if(self.savefig==True):                
                plt.savefig(self.savepath+self.runname+'_'+'Sal_WOA_Taylor'+'_'+str(years[0])+'to'+str(years[1])+'.png', dpi = 300, bbox_inches='tight')
            plt.show(block=False)  
        


In [ ]:
### TESTING ###
if __name__ == "__main__":
    
    # run specification -------------------------------------------------------------------------------------
    runid      =  'fesom'
    runname = runid
    layerwise    = False

    resultpath = '/work/ollie/mozeisin/results/f2r1.2/' + 'mo5'
    savepath = '/home/ollie/mozeisin/evaluation/mo_files/'

    meshpath = '/work/ollie/mozeisin/mesh/mesh_fesom2.0/core2_meanz'

    # period of analysis ------------------------------------------------------------------------------------

    first_year = 1948
    last_year  = 1957
    years = [first_year, last_year]

    # WOA ------------------------------------------------------------------------------------
    ncpath =  '/work/ollie/projects/MarESys/evaluation/woa18_decav_s00_01.nc'

    # now test:
    test = SALcomp(runid,resultpath,savepath,meshpath,ncpath,first_year,last_year,
               get_overview = False,
               use_temp_mask = False,
               mapproj='pc',
               layerwise=False,savefig=False)